In [5]:
import pandas as pd
import numpy as np

In [6]:
# Основний датасет 
df_calls = pd.read_csv("bd_lab_large_Lviv.csv",encoding="cp1251")


# Датасет із зупинками
df_stops = pd.read_table("untitled.txt", encoding = "cp1251", sep = ";")


In [7]:
# date to datetime
df_calls["event_start_date"] = df_calls["event_start_date"].apply(lambda x: x.lstrip())
df_calls["event_start_date"] = pd.to_datetime(df_calls["event_start_date"], format='%d%b%Y:%H:%M:%S')


In [11]:
# Lviv center coordinates (central point)
center_lat, center_lon = 49.84, 24.03  # Lviv
#center_lat, center_lon = 48.28, 25.94  # Chernivtsi


## calculate length in km of 1 degree for latitude and longitude
a = 6378.137; b = 6356.752 # semiaxis of Earth ellipsoid in km
e2 = (a**2 - b**2) / a**2 # eccentricity of Earth

# Convert degree to radians
def deg_to_rad(phi):
    return np.pi * phi / 180

# one degree of latitude and longitude in km
deg_lat_in_km = np.pi / 180 * a * (1-e2) /\
            (1 - e2*np.sin(deg_to_rad(center_lat)))**1.5
deg_lon_in_km = np.pi / 180 * a * np.cos(deg_to_rad(center_lat)) /\
             np.sqrt(1 - e2*np.sin(deg_to_rad(center_lat))**2 )

print("1° of latitude = {0:5.2f} km".format(deg_lat_in_km)) 
print("1° of longitude = {0:5.2f} km".format(deg_lon_in_km))


1° of latitude = 111.43 km
1° of longitude = 71.93 km


In [12]:
#Convert distance in km to degrees
# for latitude (y-coordinate)
def km_to_lat(x):
    '''x - distance in km; return distance in degrees of latitude'''
    return x / deg_lat_in_km

# for longitude (x-coordinate)
def km_to_lon(x):
    '''x - distance in km; return distance in degrees of latitude'''
    return x / deg_lon_in_km

# Invert km to lat
def lat_to_km(deg):
    '''deg - distance in latitude degrees; return distance in km'''
    return deg * deg_lat_in_km

def lon_to_km(deg):
    '''deg - distance in latitude degrees; return distance in km'''
    return deg * deg_lon_in_km



In [15]:
# Distance (in km) between two points on map
def dist(point_1, point_2):
    '''find distance in km between 2 points (lat, lon)'''
    dlat = point_1[0] - point_2[0]
    dlon = point_1[1] - point_2[1]
    dlat_km, dlon_km = lat_to_km(dlat), lon_to_km(dlon)
    return np.hypot(dlat_km, dlon_km)

## перевірка Львів-Кропивницький - 616 км
#find_dist((49.84, 24.03), (48.5, 32.27))
    
def closest_stop(coord_client):
    '''find stop at min distance from coord_client'''
    # create auxiliary column with distance for current client
    df_stops["dist"] = dist((df_stops["stop_lat"], df_stops["stop_lon"]), coord_client)
    ind = np.argmin(df_stops["dist"])  # index of stop with minimum distance
    del df_stops["dist"] # remove auxiliary column
    stop_id = df_stops["stop_id"].iloc[ind]
    return stop_id


In [26]:
#applying function to dataset
df_calls['closest_stop'] = df_calls.apply(lambda row: closest_stop((row['LAT'], row['LON'])), axis=1)


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)


In [21]:
df_calls['weekday'] = df_calls['event_start_date'].dt.dayofweek

In [22]:
#adding day type
def func(row):
    if row['weekday'] in (0,1,2,3):
        return 1
    elif row['weekday'] == '5' and row['time_hour'] in (20,21,22,23):
        return 2
    elif row['event_start_date'] == '2017-08-24' or row['event_start_date'] == '2017-06-28' or row['event_start_date'] == '2017-06-05':
        return 3
    elif row['event_start_date'] == '2017-08-23' or row['event_start_date'] == '2017-06-27' and row['time_hour'] in (20,21,22,23):
        return 3
    else:
        return 2


df_calls['day_type'] = df_calls.apply(func, axis=1)

In [25]:
df_calls

hash_number_A  hash_tariff   event_type    event_sub  \
0               148731       239056  network_ser      content   
1               148731       239056  network_ser      content   
2               148731       239056  network_ser    omo_voice   
3               148731       239056  network_ser      omo_sms   
4               148731       239056  network_ser      omo_sms   
5               148731       239056  network_ser      omo_sms   
6               148731       239056  network_ser    omo_voice   
7               148731       239056  network_ser    omo_voice   
8               148731       239056  network_ser         data   
9               148731       239056  network_ser         data   
10              148731       239056  network_ser         data   
11              148731       239056  network_ser         data   
12              148731       239056  network_ser         data   
13              148731       239056  network_ser         data   
14              148731       239056  network_ser         data   
15              148731       239056  network_ser         data   
16              148731       239056  network_ser         data   
17              148731       239056  network_ser         data   
18              148731       239056  network_ser         data   
19              148731       239056  network_ser         data   
20              148731       239056  network_ser         data   
21              148731       239056  network_ser         data   
22              148731       239056  network_ser         data   
23              148731       239056  network_ser         data   
24              148731       239056  network_ser         data   
25              148731       239056  network_ser         data   
26              148731       239056  network_ser         data   
27              148731       239056  network_ser         data   
28              148731       239056  network_ser         data   
29              148731       239056  network_ser         data   
...                ...          ...          ...          ...   
3937252        2827380       187131  network_ser      content   
3937253        2827380       187131  network_ser      content   
3937254        2827380       187131  network_ser      content   
3937255        2827380       187131  network_ser      content   
3937256        2827380       187131  network_ser      content   
3937257        2827380       187131  network_ser      content   
3937258        2827380       187131  network_ser      content   
3937259        2827380       187131  network_ser      content   
3937260        2827380       187131  network_ser      content   
3937261        2827433       217307  network_ser      omo_sms   
3937262        2827433       217307  network_ser      omo_sms   
3937263        2827433       217307  network_ser      omo_sms   
3937264        2827433       217307  network_ser         data   
3937265        2827433       217307  network_ser         data   
3937266        2827433       217307  network_ser         data   
3937267        2827433       217307  network_ser         data   
3937268        2827449       238465  network_ser      content   
3937269        2827449       238465  network_ser      content   
3937270        2827449       238465  network_ser  onnet_voice   
3937271        2827449       238465  network_ser  onnet_voice   
3937272        2827449       238465  network_ser    onnet_sms   
3937273        2827449       238465  network_ser    onnet_sms   
3937274        2827449       238465  network_ser    onnet_sms   
3937275        2827449       238465  network_ser    onnet_sms   
3937276        2827449       238465  network_ser  onnet_voice   
3937277        2827449       238465  network_ser  onnet_voice   
3937278        2827449       238465  network_ser  onnet_voice   
3937279        2827449       238465  network_ser  onnet_voice   
3937280        2827449       238465  network_ser  onnet_voice   
3937281        2827449       238465  network

In [ ]:
#amount of people on stop group by hour
most_visited_stops = df_calls.groupby(['closest_stop', 'hour']).size()
most_visited_stops .rename(columns={'0':'hour'})
most_visited_stops.to_csv('most_visited_stops.csv', header=True)